In [55]:
import pandas as pd

In [ ]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [4]:
documents[2]

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'}

In [5]:
df = pd.DataFrame(documents, columns=['course', 'section', 'question', 'text'])

# Vector spaces

- turn the docs into vectors
- term-document matrix:
    - rows: documents
    - columns: words/tokens
- bag of words:
    - word order is lost
    - spars matrix

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
cv = CountVectorizer(min_df=5)

In [8]:
cv.fit(df.text)

CountVectorizer(min_df=5)

In [9]:
cv.get_feature_names_out()

array(['01', '02', '03', ..., 'youtube', 'zip', 'zoomcamp'],
      shape=(1524,), dtype=object)

In [10]:
doc_examples = [
    "Course starts on 15th Jan 2024",
    "Prerequisites listed on GitHub",
    "Submit homeworks after start date",
    "Registration not required for participation",
    "Setup Google Cloud and Python before course"
]

In [11]:
cv = CountVectorizer()
cv.fit(doc_examples)

CountVectorizer()

In [12]:
cv.get_feature_names_out()

array(['15th', '2024', 'after', 'and', 'before', 'cloud', 'course',
       'date', 'for', 'github', 'google', 'homeworks', 'jan', 'listed',
       'not', 'on', 'participation', 'prerequisites', 'python',
       'registration', 'required', 'setup', 'start', 'starts', 'submit'],
      dtype=object)

In [13]:
X = cv.transform(doc_examples)

In [14]:
pd.DataFrame(X.todense(), columns=cv.get_feature_names_out()).T

,0,1,2,3,4
15th,1,0,0,0,0
2024,1,0,0,0,0
after,0,0,1,0,0
and,0,0,0,0,1
before,0,0,0,0,1
cloud,0,0,0,0,1
course,1,0,0,0,1
date,0,0,1,0,0
for,0,0,0,1,0
github,0,1,0,0,0


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

cv = TfidfVectorizer(stop_words='english', min_df=5)
X = cv.fit_transform(df.text)

In [16]:
X

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 23808 stored elements and shape (948, 1333)>

In [17]:
query = "I just discovered the course, is it to late to join?"

q = cv.transform([query])
q.toarray()

array([[0., 0., 0., ..., 0., 0., 0.]], shape=(1, 1333))

In [18]:
from sklearn.metrics.pairwise import cosine_similarity

In [19]:
score = cosine_similarity(X, q).flatten()

In [20]:
import numpy as np
np.argsort(score)[-5:]

array([ 22, 448, 449, 440,   0])

In [21]:
df.iloc[449].text

'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.'

In [22]:
fields = ['section', 'question', 'text']

In [33]:
matrices = {}
vectorizers = {}

for f in fields:
    cv = TfidfVectorizer(stop_words='english', min_df=5)
    X = cv.fit_transform(df[f])
    matrices[f] = X
    vectorizers[f] = cv

In [34]:
vectorizers

{'section': TfidfVectorizer(min_df=5, stop_words='english'),
 'question': TfidfVectorizer(min_df=5, stop_words='english'),
 'text': TfidfVectorizer(min_df=5, stop_words='english')}

In [35]:
n = len(df)

In [36]:
score = np.zeros(n)

query = "I just discovered the course, is it to late to join?"

boosts = {
    'question': 3
    # 'text': 0.5
}

for f in fields:
    q = vectorizers[f].transform([query])
    X = matrices[f]

    f_score = cosine_similarity(X, q).flatten()

    boost = boosts.get(f, 1.0)
    
    score = score + boost * f_score

In [37]:
filters = {
    'course': 'data-engineering-zoomcamp'
}

In [38]:
for field, value in filters.items():
    mask = (df[field] == value).astype(int).values
    score *= mask

score

array([3.52985023, 3.49512426, 2.70735166, 2.96614194, 3.49512426,
       3.49512426, 1.93689291, 3.67069698, 2.67242848, 3.49512426,
       3.10198469, 2.46096752, 0.49512426, 0.49512426, 0.49512426,
       0.59193348, 0.49512426, 2.63772182, 0.57041627, 0.49512426,
       0.49512426, 0.49512426, 0.79499188, 0.60033101, 0.49512426,
       0.49512426, 0.49512426, 0.76959902, 0.62340833, 0.49512426,
       0.49512426, 0.49512426, 0.49512426, 1.78972334, 3.49512426,
       1.72080809, 0.49512426, 0.49512426, 0.49512426, 0.52668735,
       0.54427244, 2.00115141, 0.49512426, 0.53842198, 0.        ,
       0.        , 0.        , 0.        , 0.02804374, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.06739038, 0.        , 0.00980845,
       0.        , 0.        , 0.        , 0.        , 0.05820102,
       0.        , 0.        , 0.        , 0.        , 0.     

In [39]:
idx = np.argsort(-score)[:5]

In [40]:
df.iloc[idx]

,course,section,question,text
7,data-engineering-zoomcamp,General course-related questions,Course - Can I follow the course after it fini...,"Yes, we will keep all the materials after the ..."
0,data-engineering-zoomcamp,General course-related questions,Course - When will the course start?,The purpose of this document is to capture fre...
4,data-engineering-zoomcamp,General course-related questions,Course - What can I do before the course starts?,You can start by installing and setting up all...
1,data-engineering-zoomcamp,General course-related questions,Course - What are the prerequisites for this c...,GitHub - DataTalksClub data-engineering-zoomca...
5,data-engineering-zoomcamp,General course-related questions,Course - how many Zoomcamps in a year?,"There are 3 Zoom Camps in a year, as of 2024. ..."


# Putting it all together
Let's create a class for us to use:

In [41]:
class TextSearch:

    def __init__(self, text_fields):
        self.text_fields = text_fields
        self.matrices = {}
        self.vectorizers = {}

    def fit(self, records, vectorizer_params={}):
        self.df = pd.DataFrame(records)

        for f in self.text_fields:
            cv = TfidfVectorizer(**vectorizer_params)
            X = cv.fit_transform(self.df[f])
            self.matrices[f] = X
            self.vectorizers[f] = cv

    def search(self, query, n_results=10, boost={}, filters={}):
        score = np.zeros(len(self.df))

        for f in self.text_fields:
            b = boost.get(f, 1.0)
            q = self.vectorizers[f].transform([query])
            s = cosine_similarity(self.matrices[f], q).flatten()
            score = score + b * s

        for field, value in filters.items():
            mask = (self.df[field] == value).values
            score = score * mask

        idx = np.argsort(-score)[:n_results]
        results = self.df.iloc[idx]
        return results.to_dict(orient='records')

In [42]:
index = TextSearch(
    text_fields=['section', 'question', 'text']
)
index.fit(documents)

index.search(
    query='I just signed up. Is it too late to join the course?',
    n_results=5,
    boost={'question': 3.0},
    filters={'course': 'data-engineering-zoomcamp'}
)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineerin

In [43]:
from sklearn.decomposition import TruncatedSVD

X = matrices['text']
cv = vectorizers['text']

In [48]:
svd = TruncatedSVD(n_components=16)
X_emb = svd.fit_transform(X)

In [49]:
X_emb[0]

array([ 0.09652983, -0.08218271, -0.10105201, -0.07734852,  0.0688718 ,
       -0.06153214,  0.02254414, -0.12836767,  0.2555874 ,  0.28587219,
        0.0610339 ,  0.02977698, -0.10024032,  0.03285606,  0.04323393,
       -0.00086647])

In [50]:
query = 'I just signed up. Is it too late to join the course?'

Q = cv.transform([query])
Q_emb = svd.transform(Q)
Q_emb[0]

array([ 0.05790204, -0.03858159, -0.05586839, -0.02628251,  0.04000945,
       -0.06233964,  0.01293557, -0.0799097 ,  0.1738801 ,  0.17853372,
        0.05463586,  0.0371027 , -0.08126072,  0.02088644,  0.04621913,
        0.02220388])

In [56]:
np.dot(X_emb[0], Q_emb[0])

np.float64(0.14430907743222587)

In [58]:
score = cosine_similarity(X_emb, Q_emb).flatten()
idx = np.argsort(-score)[:10]
df.loc[idx]

,course,section,question,text
449,machine-learning-zoomcamp,General course-related questions,The course has already started. Can I still jo...,"Yes, you can. You won’t be able to submit some..."
764,machine-learning-zoomcamp,Projects (Midterm and Capstone),What If I submitted only two projects and fail...,If you have submitted two projects (and peer-r...
0,data-engineering-zoomcamp,General course-related questions,Course - When will the course start?,The purpose of this document is to capture fre...
11,data-engineering-zoomcamp,General course-related questions,Certificate - Can I follow the course in a sel...,"No, you can only get a certificate if you fini..."
436,machine-learning-zoomcamp,General course-related questions,Is it going to be live? When?,"The course videos are pre-recorded, you can st..."
450,machine-learning-zoomcamp,General course-related questions,When does the next iteration start?,The course is available in the self-paced mode...
814,mlops-zoomcamp,+-General course questions,What if my answer is not exactly the same as t...,Please choose the closest one to your answer. ...
7,data-engineering-zoomcamp,General course-related questions,Course - Can I follow the course after it fini...,"Yes, we will keep all the materials after the ..."
13,data-engineering-zoomcamp,General course-related questions,Office Hours - I can’t attend the “Office hour...,Yes! Every “Office Hours” will be recorded and...
451,machine-learning-zoomcamp,General course-related questions,Can I submit the homework after the due date?,"No, it’s not possible. The form is closed afte..."


In [59]:
from sklearn.decomposition import NMF

nmf = NMF(n_components=16)
X_emb = nmf.fit_transform(X)
X_emb[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.00061215, 0.        , 0.26450066, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        ])

In [60]:
Q = cv.transform([query])
Q_emb = svd.transform(Q)
Q_emb[0]

array([ 0.05790204, -0.03858159, -0.05586839, -0.02628251,  0.04000945,
       -0.06233964,  0.01293557, -0.0799097 ,  0.1738801 ,  0.17853372,
        0.05463586,  0.0371027 , -0.08126072,  0.02088644,  0.04621913,
        0.02220388])

In [63]:
score = cosine_similarity(X_emb, Q_emb).flatten()
idx = np.argsort(-score)[:10]
df.loc[idx]

,course,section,question,text
810,mlops-zoomcamp,+-General course questions,Format for questions: [Problem title],MLOps Zoomcamp FAQ\nThe purpose of this docume...
807,machine-learning-zoomcamp,Miscellaneous,Is use of libraries like fast.ai or huggingfac...,"Yes, it’s allowed (as per Alexey).\nAdded By R..."
448,machine-learning-zoomcamp,General course-related questions,I’m new to Slack and can’t find the course cha...,Here’s how you join a in Slack: https://slack....
816,mlops-zoomcamp,+-General course questions,Can I still graduate when I didn’t complete ho...,"In order to obtain the certificate, completion..."
797,machine-learning-zoomcamp,Miscellaneous,I may end up submitting the assignment late. W...,Depends on whether the form will still be open...
135,data-engineering-zoomcamp,Module 1: Docker and Terraform,GCP VM - Is it necessary to use a GCP VM? When...,The reason this video about the GCP VM exists ...
808,machine-learning-zoomcamp,Miscellaneous,"Flask image was built and tested successfully,...",The TF and TF Serving versions have to match (...
29,data-engineering-zoomcamp,General course-related questions,Besides the “Office Hour” which are the live z...,We will probably have some calls during the Ca...
765,machine-learning-zoomcamp,Projects (Midterm and Capstone),I did the first two projects and skipped the l...,Yes. You only need to review peers when you su...
8,data-engineering-zoomcamp,General course-related questions,Course - Can I get support if I take the cours...,"Yes, the slack channel remains open and you ca..."


# BERT

In [64]:
pip install transformers tqdm torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 38.6 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 53.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [65]:
import torch
from transformers import BertModel, BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")
model.eval()  # Set the model to evaluation mode if not training

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

We need:

tokenizer - for turning text into vectors
model - for compressing the text into embeddings
First, we tokenize the text

In [68]:
texts = [
    "Yes, we will keep all the materials after the course finishes.",
    "You can follow the course at your own pace after it finishes"
]
encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
encoded_input

{'input_ids': tensor([[  101,  2748,  1010,  2057,  2097,  2562,  2035,  1996,  4475,  2044,
          1996,  2607, 12321,  1012,   102],
        [  101,  2017,  2064,  3582,  1996,  2607,  2012,  2115,  2219,  6393,
          2044,  2009, 12321,   102,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]])}

In [69]:
with torch.no_grad():  # Disable gradient calculation for inference
    outputs = model(**encoded_input)
    hidden_states = outputs.last_hidden_state

In [70]:
hidden_states.shape

torch.Size([2, 15, 768])

In [71]:
hidden_states[0]

tensor([[ 0.1010,  0.0181,  0.1303,  ..., -0.2932,  0.1863,  0.6615],
        [ 1.0608, -0.1242,  0.1370,  ..., -0.1605,  1.0429,  0.3532],
        [ 0.1802,  0.0776,  0.3941,  ..., -0.1379,  0.5974,  0.1704],
        ...,
        [ 0.4738, -0.0184,  0.2186,  ..., -0.0013, -0.0833, -0.2170],
        [ 0.6516,  0.1216, -0.2494,  ...,  0.1557, -0.5632, -0.4310],
        [ 0.7164,  0.2157, -0.0281,  ...,  0.2281, -0.6725, -0.3245]])

In [72]:
hidden_states[0].shape

torch.Size([15, 768])

In [73]:
sentence_embeddings = hidden_states.mean(dim=1)
sentence_embeddings.shape

torch.Size([2, 768])

In [74]:
sentence_embeddings

tensor([[ 0.3600, -0.1607,  0.3545,  ...,  0.0429,  0.0348, -0.0382],
        [ 0.1785, -0.5000,  0.2528,  ..., -0.1141, -0.3361,  0.4110]])

In [78]:
class A:
    lst = []
    def __init__(self, *args):
        self.lst.append(args)

a = A(1, 2, 3)
b = A(4, 5)
print(a.lst)
print(b.lst)

b.lst = []

print(a.lst)
print(b.lst)

[(1, 2, 3), (4, 5)]
[(1, 2, 3), (4, 5)]
[(1, 2, 3), (4, 5)]
[]
